# In this case, sequential model is appropriate

- Exactly one input tensor and one output tensor

# In this case, sequential model is not appropriate

- Model has multiple inputs or outputs
- Any of layers has multiple inputs or outputs
- Layer sharing
- non-linear topology

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Start example
## Define Sequential model with 3 layers


In [ ]:
model = keras.Sequential([
                          layers.Dense(2,activation = 'relu',name = "layer1"),
                          layers.Dense(3,activation = 'relu',name = "layer2"),
                          layers.Dense(4,name = "layer3")
])

## Call model on a test input

In [ ]:
x = tf.ones((3,3))
y = model(x)

# Access to layers of model

In [ ]:
model.layers

# Create a Sequential model incrementally

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2,activation='relu'))
model.add(layers.Dense(3,activation='relu'))
model.add(layers.Dense(4))

# Remove layers

In [ ]:
model.pop()
model.layers

# Sequential constructor accepts a name argument
- Userful to annotate TensorBoard graphs

In [ ]:
model =keras.Sequential(name="my_sequential")
model.add(layers.Dense(2,activation='relu',name = 'layer1'))
model.add(layers.Dense(3,activation='relu',name = 'layer2'))
model.add(layers.Dense(4,name = 'layer3'))

# When you instantiate a Sequential model without an input shape, it isn't "built" 
- it has no weights
- model.weights results in an error starting.


In [ ]:
model = keras.Sequential([layers.Dense(2,activation = "relu"),
                          layers.Dense(3,activation = "relu"),
                          layers.Dense(4)])

In [ ]:
model.weights

In [ ]:
model.summary()

In [ ]:
x = tf.ones((1,4))
y = model(x)
model.summary()

# Predefined the input shape
- useful when building a Sequential model incrementally to be able to display the summary of the model.
- I should start my model by passing an input object to my model, so that it knows its input shape from the start.
- Input object is not displayed as part of model.layers, since it isn't a layer.
- Models with predefined input shape always have weights and a defined output shape.


In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2,activation='relu'))

model.summary()

In [ ]:
model.layers

## A simple alternative


In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2,activation = 'relu',input = (4,)))

model.summary()

# A common debugging workflow
- add + summary -> incrementally stack layers and frequently print model summary.
- Userful


In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape = (250,250,3)))
model.add(layers.Conv2D(32,5,strides=2,activation = 'relu'))

In [ ]:
model.summary()

In [ ]:
model.add(layers.Conv2D(32,3,activation = 'relu'))
model.summary()

In [ ]:
model.add(layers.MaxPooling2D(3))
model.summary()

# Q. Why is the output shape Maxpooling 40 40 32 ? (0114)

In [ ]:
model.add(layers.Conv2D(32,3,activation = 'relu'))
model.add(layers.Conv2D(32,3,activation = 'relu'))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32,3,activation = 'relu'))
model.add(layers.Conv2D(32,3,activation = 'relu'))
model.add(layers.MaxPooling2D(3))

In [ ]:
model.summary()

In [ ]:
model.add(layers.GlobalAveragePooling2D())

In [ ]:
model.add(layers.Dense(10))

In [ ]:
model.summary()

# Feature extraction with a sequential model

- Once built, it acts like a Functional api model. = Every layer has an input and output.
- For example, creating a model that extracts the outputs of all intermediate layers in a Sequentail model.



In [ ]:
initial_model = keras.Sequential([
                                  keras.Input(shape =(250,250,3)),
                                  layers.Conv2D(32,5,strides=2,activation="relu"),
                                  layers.Conv2D(32,3,activation="relu"),
                                  layers.Conv2D(32,3,activation="relu")
])

feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs = [layer.output for layer in initial_model.layers]
)

In [ ]:
x = tf.ones((1,250,250,3))
features = feature_extractor(x)
features

# Q. I understand the fact that outputs of intermediate layers can be extracted. I want to know how to use keras.model and whether training is conducted by featrue_extractor or not.

## Extracting feature from only one layer

In [ ]:
initial_model = keras.Sequential([
                                  keras.Input(shape=(250,250,3)),
                                  layers.Conv2D(32,5,strides=2,activation='relu'),
                                  layers.Conv2D(32,3,activation="relu",name = "my_intermediate_layer"),
                                  layers.Conv2D(32,3,activation = "relu")
                                  
])

feature_extractor = keras.Model(
    inputs = initial_model.inputs,
    outputs = initial_model.get_layer(name="my_intermediate_layer").output
)

x = tf.ones((1,250,250,3))
features = feature_extractor(x)

# Transfer learning with a Sequential model

- Two approaches
  - Load pre-trained weights and freeze
  - Use a sequential model to stack a pre-trained model.

In [ ]:
model = keras.Sequential([
                          keras.Input(shape=(784)),
                          layers.Dense(32,activation = 'relu'),
                          layers.Dense(32,activation = 'relu'),
                          layers.Dense(32,activation = 'relu'),
                          layers.Dense(10)
])

model.load_weights(...)

for layer in model.layers[:-1]:
  layer.trainalbe = False
model.compile()
model.fit()

In [ ]:
base_model = keras.applications.Xception(
    weights = "imagenet",
    include_top= False,
    pooling = 'avg'

)

base_model.trainalbe = False

model = keras.Sequential([
                          base_model,
                          layers.Dense(1000)
])

model.compile()
model.fit()